# PYTHON CHECKER GAME PLAYED WITH SMART COMPUTER AI
## Creator: Lam Tran *(461300)*, Tri Kieu *(464376)*

This purpose of this project/assignment is to create game-playing interface called checker (or draughts) using Pygame package that played with created with a programmed AI computer. AI computer can be made with the method of MinMax algorithms which mostly used in making board game interfaces. The first thing to initialize a constant value for the size of a board, checker pieces, and each square from the board. Addtionally, it is also important to give the checker game interface some color as well using RGB color code.

In [19]:
import pygame

# Initialize the size of Board
WIDTH, HEIGHT = 600,600
ROW, COL = 8,8

# Initialize the size of Square
SQUARE_SIZE = HEIGHT // ROW

# COLOR
BLACK = (0,0,0)
LIGHTBROWN = (221,189,137)
DARKBROWN = (140,93,54)
WHITE = (225,225,225)
GREY = (128, 128, 128)
GREEN = (0, 255, 0)

# Load King
CROWN = pygame.transform.scale(pygame.image.load('king_master.png'),(40,20))

### Step 1: Creating checker pieces
This class *Piece* will create each and every checker pieces on the board and gives them all the appropriate location of board at the start of the game. It also rewrite the location of the pieces when the player or AI start to make a move. 

In [20]:
class Piece:
    
    #Private variables to draw checker pieces
    PADDING = 8
    OUTLINE = 2
    
    def __init__(self, row, col, color):
        self.row = row
        self.col = col
        self.color = color
        self.isKing = False
        self.x_coord = 0
        self.y_coord = 0
        self.piece_location()

    #Give each pieces a proper location at the start or current state of a game
    def piece_location(self):
        self.x_coord = SQUARE_SIZE * self.col + SQUARE_SIZE // 2
        self.y_coord = SQUARE_SIZE * self.row + SQUARE_SIZE // 2

    # Draw one checker piece when this function is called 
    def draw_piece(self, win):
        radius = SQUARE_SIZE//2 - self.PADDING
        outline = radius + self.OUTLINE
        pygame.draw.circle(win, GREY, (self.x_coord, self.y_coord), outline)
        pygame.draw.circle(win, self.color, (self.x_coord, self.y_coord), radius)

        if self.isKing:
            win.blit(CROWN, (self.x_coord - CROWN.get_width() // 2,self.y_coord - CROWN.get_height() //2))
    
    # Become a king
    def become_king (self):
        self.isKing = True

    # Move a piece
    def move(self,row,col):
        self.row = row
        self.col = col
        self.piece_location()

### Step 2: Creating checker board game
Within the class called *"Board"*, the main program will display the board on application simply by calling the *display_board()* function on *main* class with a required parameter **WINDOW** that had a blank interface for a function *draw_board()* to draw. Additionally, class *Board* data (variable) about the current state of board game, the remaining of normal or king pieces on a board. 

In [21]:
class Board:
    def __init__(self):
        self.board = []
        self.chosen_piece = 0
        self.white_left = self.black_left = 12
        self.white_king = self.black_king = 0
        self.add_pieces_to_board()
        

    # First: Fill the whole board with Dark Brown color.
    # Second: Then the light brown square is placed. It starts from the top left (coordinate (0,0)).
    # cor_x, cor_y shows where to draw the square (x and y coordinate).
    def draw_board(self, win):
        win.fill(DARKBROWN)
        for row in range (ROW):
            for col in range(row % 2, ROW ,2):
                cor_x = row * SQUARE_SIZE
                cor_y = col * SQUARE_SIZE
                pygame.draw.rect(win,LIGHTBROWN, (cor_x,cor_y,SQUARE_SIZE,SQUARE_SIZE))

    # Adding all the checker pieces on the board at specific coordination
    # In terms of specific coordination, it shows how and where the pieces place at the start of a game
    # 0 represent empty space in the board game, whereas object from class Piece will put in that square
    def add_pieces_to_board(self):
        for row in range(ROW):
            self.board.append([])
            for col in range(COL):
                if col % 2 == ((row + 1) % 2):
                    if row < 3:
                        self.board[row].append(Piece(row, col, BLACK))
                    elif row > 4:
                        self.board[row].append(Piece(row, col, WHITE))
                    else:
                        self.board[row].append(0)
                else:
                    self.board[row].append(0)
    
    # Display the board along with other checker pieces at the start of a game
    def display_board(self, win):
        self.draw_board(win)
        for row in range(ROW):
            for col in range(COL):
                checker_piece = self.board[row][col]
                if checker_piece != 0: # If it is not empty (!=0), draw the piece in that coordination
                    checker_piece.draw_piece(win) 

    # Move the piece to a specific location on a board
    def move_piece(self,piece,row,col):
        self.board[piece.row][piece.col], self.board[row][col] = self.board[row][col], self.board[piece.row][piece.col]
        piece.move(row, col)

        if row == ROW - 1 or row == 0:
            piece.become_king()
            if piece.color == WHITE:
                self.white_king += 1
            else:
                self.black_king += 1 

    #Get the position of the selected piece on a board
    def get_piece_position(self,row,col):
        return self.board[row][col]
    
    #Two private algorithm functions that move the pieces left or right diagonally
    #Also, it check to see if there is an opponent piece to jump, possibly double/triple jump as well
    def _left_movement(self, start, stop, step, color, left_col, jumped=[]):
        moves = {}
        last = []

        for row in range(start,stop,step):
            if left_col < 0:
                break
            
            current = self.board[row][left_col]
            if current == 0:
                if jumped and not last:
                    break
                elif jumped:
                    moves[(row,left_col)] = last + jumped
                else:
                    moves[(row,left_col)] = last
                
                if last:
                    if step == -1:
                        _row_ = max(row-3, 0)
                    else:
                        _row_ = min(row+3, ROW)
                    moves.update(self._left_movement(row+step, _row_, step, color, left_col-1, jumped=last))
                    moves.update(self._right_movement(row+step, _row_, step, color, left_col+1, jumped=last))
                break
            elif current.color == color:
                break
            else:
                last = [current]
            
            left_col -= 1
        return moves

    def _right_movement(self, start, stop, step, color, right_col, jumped=[]):
        moves = {}
        last = []

        for row in range(start,stop,step):
            if right_col >= COL:
                break
            
            current = self.board[row][right_col]
            if current == 0:
                if jumped and not last:
                    break
                elif jumped:
                    moves[(row,right_col)] = last + jumped
                else:
                    moves[(row,right_col)] = last
                
                if last:
                    if step == -1:
                        _row_ = max(row-3, 0)
                    else:
                        _row_ = min(row+3, ROW)
                    moves.update(self._left_movement(row+step, _row_, step, color, right_col-1, jumped=last))
                    moves.update(self._right_movement(row+step, _row_, step, color, right_col+1, jumped=last))
                break
            elif current.color == color:
                break
            else:
                last = [current]
            
            right_col += 1
        return moves

    #Get all possible move of each pieces depending on the current state of the game or player's turn
    #Additionally, it uses the algorithm from 2 functions that control the movement of the pieces on the board
    def get_possible_moves(self, piece):
        moves = {}
        move_left = piece.col-1
        move_right = piece.col+1
        row = piece.row

        if piece.color == WHITE or piece.isKing:
            moves.update(self._left_movement(row-1, max(row-3, -1), -1, piece.color, move_left))
            moves.update(self._right_movement(row-1, max(row-3, -1), -1, piece.color, move_right))
        if piece.color == BLACK or piece.isKing:
            moves.update(self._left_movement(row+1, min(row+3, ROW), 1, piece.color, move_left))
            moves.update(self._right_movement(row+1, min(row+3, ROW), 1, piece.color, move_right))

        return moves

    # Remove the checker pieces from the board
    def remove_piece(self, pieces):
        for piece in pieces:
            self.board[piece.row][piece.col] = 0
            if piece != 0:
                if piece.color == WHITE:
                    self.white_left -= 1
                else:
                    self.black_left -= 1

    # This function set the message of the winner of checkers
    def winner(self):
        if self.white_left <= 0:
            return 'Black player is the winner!!'
        elif self.black_left <= 0:
            return 'White player is the winner!!'
        return None

### Step 3: Handling the game
The class called *'Game'* will start the game, handle the player's turn and movement of each pieces, and determine the winner

In [22]:
class Game:
    def __init__(self, win):
        self.reset()
        self.win = win
    
    #Always update the current to latest state of the game
    def update_game(self):
        self.board.display_board(self.win)
        self.highlight_moves(self.possible_moves)
        pygame.display.update()

    #This function called to reset the game
    def reset(self):
        self.selected_piece = None
        self.board = Board()
        self.player_turn = WHITE
        self.possible_moves = {}

    #Switch players when it is their turn
    def switch_turn(self):
        self.possible_moves = {}
        if self.player_turn == WHITE:
            self.player_turn = BLACK
        else:
            self.player_turn = WHITE
    
    #Move the piece on the board, return True if the move is valid (Note: a private function of class Game)
    def _piece_to_move(self, row, col):
        piece = self.board.get_piece_position(row, col)
        
        #If the player had selected the move that is empty and within possible move, move that piece on the board
        if self.selected_piece and piece == 0 and (row, col) in self.possible_moves:
            self.board.move_piece(self.selected_piece,row,col)
            jumped = self.possible_moves[(row, col)]

            if jumped: #If there is a opponent piece to jump, then jump it
                self.board.remove_piece(jumped)
                
            self.switch_turn()
        else:
            return False
        return True

    #A function which select the piece to move on the board
    def select_piece_to_move(self, row, col):
        if self.selected_piece:
            result = self._piece_to_move(row,col)
            #The current state of the game will not change if the select movement is invalid
            if not result:
                self.selected_piece = None
                self.select_piece_to_move(row,col) #Recurvsive function: ask the player to move again until valid
        
        piece = self.board.get_piece_position(row,col)
        #Only updated the piece when correctly select the color piece that match the player turns
        if piece != 0 and piece.color == self.player_turn:
            self.selected_piece = piece
            self.possible_moves = self.board.get_possible_moves(piece)
            return True
        return False
                
    
    # Hightlight all checker pieces possible moves on the board
    def highlight_moves(self, moves):
        for move in moves:
            row, col = move
            pygame.draw.circle(self.win, GREEN, (col*SQUARE_SIZE+SQUARE_SIZE//2,row*SQUARE_SIZE+SQUARE_SIZE//2), 15)
    
    # Get the winner of game checkers
    def get_winner(self):
        return self.board.winner()

### Step 4: Mouse Class Handling
This will receive the mouse position

In [23]:
def mouse_pos_cal_mouse(pos):
    x, y = pos
    row = y // SQUARE_SIZE
    col = x // SQUARE_SIZE
    return row, col
    

### MAIN CLASS PROGRAM
This main class is where to start the checker game after running all the classes above.  

In [24]:
class main:
    
    status = True
    FPS = 60

    WINDOW = pygame.display.set_mode((WIDTH,HEIGHT))
    pygame.display.set_caption("Checkers")

    clock = pygame.time.Clock()
    game = Game(WINDOW)
    count = 1

    while status == True:
        clock.tick(FPS)

        if game.get_winner() != None and count == 1:
            print(game.get_winner())
            count = 0

        for action in pygame.event.get():
            if action.type == pygame.QUIT:
                status = False          
            if action.type == pygame.MOUSEBUTTONDOWN:
                place = pygame.mouse.get_pos()
                row,col = mouse_pos_cal_mouse(place)
                game.select_piece_to_move(row,col)

        game.update_game()
    
    pygame.quit()



Black player is the winner!!
